In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from db.datasets import datasets
from config import system_configs
import json, os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import average_precision_score
import logging

In [6]:
with open('config/KPGrouping.json', "r") as f:
    configs = json.load(f)
    
split = 'valchart'

configs["system"]["data_dir"] = "data/extraction_data/"
configs["system"]["cache_dir"] = "data/extraction_data/cache/"

configs["system"]["dataset"] =  "Chart"
configs["system"]["snapshot_name"] = "PretrainKP"
system_configs.update_config(configs["system"])
db = datasets["Chart"](configs["db"], split)

Label file: data/clsdata(1031)/cls/annotations/chart_val.json
Loading from cache file: data/clsdata(1031)/cache/chart_val.pkl
Loading annotations into memory...
Done (t=0.14s)


In [10]:
def get_pie_center(a, b, c):
    a,b,c = np.array(a), np.array(b), np.array(c)
    ca = c - a
    cb = c - b
    # 计算向量 ca 和 cb 之间的角度的余弦值。
    cosine_angle = np.dot(ca, cb) / (np.linalg.norm(ca) * np.linalg.norm(cb))
    angle = np.arccos(cosine_angle)
    r_square = (ca**2).sum()
   # 判断向量 ca 和 cb 的叉积的符号。这一步用于确定扇形是在向量 ca 和 cb 之间还是在向量 ca 和 cb 之外。 
    if ca[0]*cb[1]-ca[1]*cb[0] >= 0:
        # 如果叉积大于或等于 0，则返回扇形的几何中心和面积（这里假设扇形是在向量 ca 和 cb 之间）。
        return (a[0]+b[0]+c[0])/3., (a[1]+b[1]+c[1])/3., 0.5 * angle * r_square
    else:
        # 否则，返回扇形外侧的几何中心和面积。
        return 2*c[0]-(a[0]+b[0]+c[0])/3., 2*c[1]-(a[1]+b[1]+c[1])/3., np.pi * r_square - 0.5 * angle * r_square

def get_grouped_points(gts, preds, chartType):
    gt_groups = []
    area = 0
        
    if chartType == 'Bar':
        # 遍历所有 ground truth 边界框。
        for bbox in gts:
            # 计算边界框的面积。
            area = np.abs((bbox[2] - bbox[0]) * (bbox[3] - bbox[1])) 
            # 如果面积为 0，则跳过这个边界框。
            if area == 0: continue
            # 将非零面积的 ground truth 添加到 gt_groups。
            gt_groups.append((bbox[:-1].reshape(-1, 2), area))
    elif chartType == 'Pie':
        for bbox in gts:
            # 处理逻辑与条形图类似，但是使用了 get_pie_center 函数来计算饼图分片的面积。
            a, b, c = (bbox[0], bbox[1]), (bbox[2], bbox[3]), (bbox[4], bbox[5])
            _, _, area = get_pie_center(a,b,c)        
            if area == 0: continue
            gt_groups.append((bbox[:-1].reshape(-1, 2), area))
    elif chartType == 'Line':
        for bbox in gts:
            # 遍历 ground truth，并使用特定的计算方式来获取面积。
            detection = np.array(bbox)
            if len(detection) <= 1: continue
            assert len(detection) % 2 == 0
            
            xs = detection[0:len(detection):2]
            ys = detection[1:len(detection):2]
            area = (max(max(xs) - min(xs), max(ys) - min(ys)) / len(detection) * 2) ** 2
            if area == 0: continue
                
            gt_groups.append((bbox.reshape(-1, 2), area))   

    pred_groups = []
    if '1' not in preds[0]: # baseline predictions
        if chartType == 'Pie':
            for pred in preds:
                pred_groups.append(np.array(pred[:-1]))
        elif chartType == 'Line':
            for pred in preds:
                pred_groups.append(np.array(pred))
        else:
            for pred in preds:
                pred_groups.append(np.array(pred).reshape(-1, 2))
    else:
        for pred in preds[2]:
            pred_groups.append(np.array(pred[2:-1]).reshape(-1, 2))
    return gt_groups, pred_groups

In [11]:
# 计算 Object Keypoint Similarity（物体关键点相似度）。这是一种常用于评估物体检测和关键点估计任务性能的指标。
def OKS(gt_p, pred_p, area):
    # 计算了两个二维点（gt_p 为 ground truth 点，pred_p 为预测点）之间的欧氏距离的平方。
    d2 = (gt_p[0] - pred_p[0]) ** 2 + (gt_p[1] - pred_p[1]) ** 2
    # 一个常数，用于调整距离的权重。这个值可以根据具体应用进行调整。
    k2 = 0.1
    # 将函数输入的 area 直接赋值给 s2，表示物体或特征的面积。面积越大，对距离的容忍度越高。
    s2 = area
    #  OKS 的计算公式。函数返回一个介于 0 和 1 之间的值，用于表示 gt_p 和 pred_p 之间的相似度。值越接近 1，表示两点越相似。
    return np.exp(d2/(s2 * k2) * (-1))

# 计算一组物体关键点（gt_ps 为 ground truth 关键点，pred_ps 为预测关键点）之间的平均 Object Keypoint Similarity（OKS）。
def GroupOKS(gt_ps, pred_ps):
    # 初始化一个空列表，用于存储每个预测点与所有 ground truth 点之间的最大 OKS。
    group_score = []
    for pred_p in pred_ps:
        # 初始化 max_score 为 0，用于存储当前预测点与所有 ground truth 点之间的最大 OKS。
        max_score = 0.
        for gt_p in gt_ps[0]:
            max_score = max(max_score, OKS(gt_p, pred_p, gt_ps[1]))
        group_score.append(max_score)
        # 计算 group_score 列表中所有 OKS 的平均值，并返回。
    return sum(group_score)/len(group_score)

def computePrecision(gt_groups, pred_groups, thres=0.75):
    count = 0
    for pred_ps in pred_groups:
        for gt_ps in gt_groups:
            if GroupOKS(gt_ps, pred_ps) > thres:
                count += 1
                break

    return count / len(pred_groups)

def computeRecall(gt_groups, pred_groups, thres=0.75):
    count = 0
    for gt_ps in gt_groups:
        for pred_ps in pred_groups:
            if GroupOKS(gt_ps, pred_ps) > thres:
                count += 1
                break

    return count / len(gt_groups)

In [4]:
with open('evaluation/KPGrouping5000.json') as f:
    prediction = json.load(f)

In [12]:
# compute 0.75
macro_P = []
macro_R = []
max_iter = db.db_inds.size

for i in tqdm(range(max_iter)):
    chartType = None
    db_ind = db.db_inds[i]
    image_file = db.image_file(db_ind)
    gts = db.detections(db_ind)
    preds = prediction[image_file.split('/')[-1]]
    if preds is None or len(preds) == 0: continue
    if len(preds) == 3 and len(preds[2]) == 0: continue
    
    print(preds)
    if gts is None or len(gts) == 0: continue
    print(gts)
    gts = gts[0] if chartType == 'Line' else gts
    if len(gts) == 0: continue
    gt_groups, pred_groups = get_grouped_points(gts, preds, chartType)

    P = computePrecision(gt_groups, pred_groups, thres=0.75)
    R = computeRecall(gt_groups, pred_groups, thres=0.75)
    
    macro_P.append(P)
    macro_R.append(R)
    

  0%|          | 2/3695 [00:00<00:02, 1292.34it/s]

[{'1': [[0.5221433639526367, 1.0, 66.86555480957031, 144.81338500976562], [0.5090314149856567, 1.0, 335.46820068359375, 166.95306396484375], [0.5082980394363403, 1.0, 335.1124572753906, 154.9727325439453], [0.5068081617355347, 1.0, 513.445556640625, 187.90850830078125], [0.5018704533576965, 1.0, 424.00164794921875, 193.54042053222656], [0.4925021529197693, 1.0, 666.405517578125, 224.15037536621094], [0.4898742139339447, 1.0, 398.6473693847656, 167.15406799316406], [0.48546725511550903, 1.0, 602.8373413085938, 223.72422790527344], [0.4747300446033478, 1.0, 513.415283203125, 198.84178161621094], [0.474124550819397, 1.0, 156.06536865234375, 113.29442596435547], [0.4688337445259094, 1.0, 602.7530517578125, 213.71917724609375], [0.46808141469955444, 1.0, 424.0223388671875, 164.97418212890625], [0.4615509808063507, 1.0, 155.94058227539062, 205.1022491455078], [0.4575537443161011, 1.0, 132.11447143554688, 144.93312072753906], [0.4562157988548279, 1.0, 67.15483856201172, 96.97804260253906], [0

NameError: name 'chartType' is not defined

In [26]:
macro_P = np.array(macro_P)
macro_R = np.array(macro_R)
macro_P_avg = macro_P[~np.isnan(macro_P)].mean()
macro_R_avg = macro_R[~np.isnan(macro_R)].mean()
print('macro_P:', macro_P_avg, " macro_R:", macro_R_avg, 'F score:', (2*macro_P_avg*macro_R_avg) / (macro_P_avg + macro_R_avg))


macro_P: 0.5838068181818181  macro_R: 0.5011583011583012 F score: 0.5393346347992855
